Input1; 
Select ([Version].[Version Name] * [Item].[Item] * [Channel].[Channel] *[Channel].[Planning Channel]* [Region].[Region] *[Region].[Planning Region] * [Location].[Location] * [PnL].[PnL] * [Demand Domain].[Demand Domain] * [Time].[Day]*[Time].[Week]* [Account].[Account] ) on row, 
({Measure.[Actual]}) on column;
Input2;
Select ([Version].[Version Name] * [Channel].[Planning Channel] * [Region].[Region] ) on row,  ({Measure.[Store Ecom Association]}) on column;
Input3; 
select (Region.[Planning Region] * Channel.[Stat Channel]) on row, () on column include memberproperties{Region.[Planning Region],DisplayName};
 
Output measure;
PandoraSR W Actuals
 
Select ([Version].[Version Name] * [Item].[Item] * [Location].[Location] * [Activity1].[Activity1] * [Time].[Week] ) on row, 
({Measure.[PandoraSR W Actuals]}) on column;

In [ ]:
# just for reference , if in future it will require

customer_group = spark.read.csv("/workspaces/my_development_codes/notebooks/data_actual_pandorasractual/region_statchannel.csv", header=True, inferSchema=True)
customer_group.show(5)
customer_group.withColumn("[Sales Domain].[Customer Group]",concat(col("`Region.[DisplayName]`"),lit("_"),col("`Channel.[Stat Channel]`"))).show(5)

+------------------------+--------------------+----------------------+
|Region.[Planning Region]|Region.[DisplayName]|Channel.[Stat Channel]|
+------------------------+--------------------+----------------------+
|                      OM|                Oman|               Non O&O|
|                      OM|                Oman|          O&O Physical|
|                      OM|                Oman|            O&O Online|
|                      PL|              Poland|               Non O&O|
|                      PL|              Poland|          O&O Physical|
+------------------------+--------------------+----------------------+
only showing top 5 rows


## Plugin Creation Started

In [ ]:
# create spark session and import necessary libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import concat, col,lit
from pyspark.sql.types import IntegerType
from pyspark.sql import functions as F

spark = SparkSession.builder \
    .appName("Actual_To_SR_Actual") \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/07/30 09:54:02 WARN Utils: Your hostname, codespaces-780a6a, resolves to a loopback address: 127.0.0.1; using 10.0.2.183 instead (on interface eth0)
25/07/30 09:54:02 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/30 09:54:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
from o9_common_utils.O9DataLake import O9DataLake 
import logging 
from pyspark.sql.functions import concat, col,lit
from pyspark.sql.types import IntegerType
from pyspark.sql import functions as F
logger = logging.getLogger("o9_logger") 

# o9datalake inputs

actual = O9DataLake.get("actual") 

ecom = O9DataLake.get("ecom") 

In [2]:
#spark inputs
ecom = spark.read.csv("/workspaces/my_development_codes/notebooks/data_actual_pandorasractual/ecom_assoc.csv", header=True, inferSchema=True)
ecom.show(2)

actual = spark.read.csv("/workspaces/my_development_codes/notebooks/data_actual_pandorasractual/actual_top_3lac.csv", header=True, inferSchema=True)
actual.show(2)

+----------------------+--------------------------+---------------+----------------------+
|Version.[Version Name]|Channel.[Planning Channel]|Region.[Region]|Store Ecom Association|
+----------------------+--------------------------+---------------+----------------------+
|    CurrentWorkingView|                O&O Online|     SK10060522|            SK10060522|
|    CurrentWorkingView|                O&O Online|     RS10037680|                BEECOM|
+----------------------+--------------------------+---------------+----------------------+
only showing top 2 rows


+----------------------+-----------+--------------------+--------------------------+---------------+------------------------+-------------------+---------+-----------------------------+-----------+-----------+-----------------+------+
|Version.[Version Name]|Item.[Item]|   Channel.[Channel]|Channel.[Planning Channel]|Region.[Region]|Region.[Planning Region]|Location.[Location]|PnL.[PnL]|Demand Domain.[Demand Domain]| Time.[Day]|Time.[Week]|Account.[Account]|Actual|
+----------------------+-----------+--------------------+--------------------------+---------------+------------------------+-------------------+---------+-----------------------------+-----------+-----------+-----------------+------+
|    CurrentWorkingView|  792352C01|Non O&O Concept S...|                   Non O&O|     RS10037680|                      RS|                All|      All|                          All|17-Jul-2023|   W29-2023|              All|     1|
|    CurrentWorkingView|  792352C01|Non O&O Concept S...|   

In [8]:
# there are 2 dataframes , ecom and actual , you have to bring the Store Ecom Association from the ecom and put into actual

# Step 1: Filter for O&O Online and select relevant columns from ecom
ecom_filtered = ecom.filter(F.col("`Channel.[Planning Channel]`") == "O&O Online") \
                   .select(F.col("`Region.[Region]`").alias("region_key"), "Store Ecom Association")

# drop "Non O&O" records from Channel.Planning Channel and group by
actual_grouped = actual.filter(actual["`Channel.[Planning Channel]`"] != "Non O&O") \
                      .groupBy("`Version.[Version Name]`","`Item.[Item]`","`Region.[Region]`","`Channel.[Planning Channel]`","`Time.[Week]`") \
                      .agg(F.sum("Actual").alias("Actual"))
actual_grouped.show(10)

# Step 3: Perform the left join and getting store ecom association for O&O Online
joined_df1 = actual_grouped.alias("a").join(
    ecom_filtered.alias("e"),
    on=F.col("a.`Region.[Region]`") == F.col("e.region_key"),
    how="left"
).withColumn("Region.[Region]",
    F.when(F.col("e.Store Ecom Association").isNotNull(), F.col("e.Store Ecom Association"))
    .otherwise(F.col("a.`Region.[Region]`"))
).drop(F.col("e.region_key"), "Store Ecom Association") \
 .withColumn("Location.[Location]", F.lit("EDC")) \
 .withColumnRenamed("Region.[Region]", "Activity1.[To Location]")\
 .withColumn("Activity1.[Activity1]",
    F.concat(F.lit("LANE_From_"), F.col("`Location.[Location]`"), F.lit("_To_"), F.col("`Activity1.[To Location]`"),F.lit("_Using_TRUCK"))
)

joined_df1.show(2,truncate=False)

output_PandoraSRWActuals = joined_df1.select(
    col("`Version.[Version Name]`"),
    col("`Item.[Item]`"),
    col("`Activity1.[Activity1]`"),
    col("`Location.[Location]`"),
    col("`Time.[Week]`"),
    col("Actual").cast(IntegerType()).alias("PandoraSR W Actuals"))

output_PandoraSRWActuals.show(5)


+----------------------+-----------+---------------+--------------------------+-----------+------+
|Version.[Version Name]|Item.[Item]|Region.[Region]|Channel.[Planning Channel]|Time.[Week]|Actual|
+----------------------+-----------+---------------+--------------------------+-----------+------+
|    CurrentWorkingView|  792352C01|     RO10052426|              O&O Physical|   W45-2023|     1|
|    CurrentWorkingView|  792352C01|     PL00000040|              O&O Physical|   W10-2025|     1|
|    CurrentWorkingView|  792352C01|     AE10053263|              O&O Physical|   W05-2025|     3|
|    CurrentWorkingView|  792352C01|     CZ10059300|              O&O Physical|   W45-2023|     1|
|    CurrentWorkingView|  792352C01|     IT10053367|              O&O Physical|   W09-2023|     1|
|    CurrentWorkingView|  792352C01|     HU10053365|              O&O Physical|   W43-2023|     1|
|    CurrentWorkingView|  792352C01|     HU10053684|              O&O Physical|   W21-2024|     1|
|    Curre

In [ ]:
# write to o9datalake
O9DataLake.put("output_PandoraSRWActuals", output_PandoraSRWActuals) 


+----------------------+-----------+-----------------------+--------------------------+-----------+------+-------------------+---------------------+
|Version.[Version Name]|Item.[Item]|Activity1.[To Location]|Channel.[Planning Channel]|Time.[Week]|Actual|Location.[Location]|Activity1.[Activity1]|
+----------------------+-----------+-----------------------+--------------------------+-----------+------+-------------------+---------------------+
|    CurrentWorkingView|  792352C01|             RO10052426|              O&O Physical|   W45-2023|     1|                EDC| LANE_From_EDC_To_...|
|    CurrentWorkingView|  792352C01|             PL00000040|              O&O Physical|   W10-2025|     1|                EDC| LANE_From_EDC_To_...|
+----------------------+-----------+-----------------------+--------------------------+-----------+------+-------------------+---------------------+
only showing top 2 rows
+----------------------+-----------+---------------------+-------------------+----